**Update**: Main working AI code has been updated on (projectName)V2. Keeping this around since it has all the functions for processing data.  

This project is designed to process lightcurve data through a simple Machine Learning program to see if it can pick up on any patterns that might indicate exoplanets are present.

We have two inital datasets:

*   List of systems we know contain exoplanets from https://exoplanet.eu/catalog/
*   List of other stars (from the Gaia star catalog  https://simbad.cds.unistra.fr/simbad/sim-fout

We will process these datasets into simple lists of star names so we can feed them into the LightKurve API which retrives the data that the space telescopes have gathered over the past two decades. https://docs.lightkurve.org/reference/index.html

For each processing step I have the program set up to store intermediate data into CSV files, this is nessesary since some intermediary steps can take several hours to run (verifying Gaia stars have lightcurve data available).


*   Exoplanet: exoplanet.eu_catalog.csv -> ExoplanetSystems.csv
*   Gaia Data: (simbad.csv renamed to GaiaDataRaw.csv) -> GaiaStarNames.csv
    *   GaiaStarNames.csv -> Gaia_LC_Systems_0_2k.csv (2.6 hours to run on 2000 stars)
    *   GaiaStarNames.csv -> Gaia_LC_Systems_2_4k.csv

Then the final code can run on that verifies exoplanet systems aren't also listed in the Gaia data. That program then creates random groups of test/training data, downloads the lightcurve data and feeds it into an AI.



In [ ]:
!pip install lightkurve

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.9/265.9 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 60.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.4/98.4 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 888.9/888.9 kB 45.1 MB/s eta 0:00:00
  Created wheel for fbpca: filename=fbpca-1.0-py3-none-any.whl size=11374 sha256=c96b5b260e64dd22a8fcc3cf3f19a64b39099ce9b8b3f38b378e053a4eef9a6e
  Stored in directory: /root/.cache/pip/wheels/3c/ea/60/8d1c9fbbc99492a1775b36a5e29c8c1ef309cc5821bd5a219d
  Created wheel for memoization: filename=memoization-0.4.0-py3-none-any.whl size=50452 sha256=20027ba6856f801df954d107de44819c162033ba2dfb0f89d7577f3c0137571c
  Stored in directory: /root/.cache/pip/wheels/3e/b8/c5/b553

In [ ]:
######################################################
#Process Exoplanet data, only make one entry per star system for confirmed planets only, and Transit Method
######################################################


import csv

#source of data: https://exoplanet.eu/catalog/#downloads-section
uniqueStarNameList = []
with open('exoplanet.eu_catalog.csv',newline='') as csvFile:
  reader = csv.DictReader(csvFile)

  for row in reader:
    if row['star_name'] not in uniqueStarNameList and row['planet_status'] == 'Confirmed' and row['detection_type'] == 'Primary Transit':
      uniqueStarNameList.append([row['star_name'], row['star_alternate_names']])
    #print(row['star_name'], row['star_distance'],row['planet_status'],row['discovered'],row['detection_type'])

print(len(uniqueStarNameList))

print(uniqueStarNameList[0])
NoDuplicates = []

#duplicate filter above was supposed to do this job but didn't so quick and dirty forloop to filter duplicates out
for x in uniqueStarNameList:
  if x not in NoDuplicates:
    NoDuplicates.append(x)

print(len(NoDuplicates))


with open('ExoplanetSystems.csv', 'w', newline='', encoding='UTF8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["star_name","star_alternate_names"])
    for x in NoDuplicates:
      #print(x)
      writer.writerow([x[0],x[1]])



#tricky part is getting data for lightcurves that don't have exoplanets in them.
#Tess has a complete list of all its targets here: https://tess.mit.edu/public/target_lists/target_lists.html

#Potential sources: https://ui.adsabs.harvard.edu/abs/2023A%26A...670A..19G/abstract
#https://ui.adsabs.harvard.edu/abs/2020yCat..36490006G/abstract

#CSV File for Gaia catalog of nearby stars from here: https://cdsarc.cds.unistra.fr/viz-bin/cat/J/A+A/649/A6#/browse 'table1c.csv'
#Found this place to get actual files out we can work with: https://simbad.cds.unistra.fr/simbad/sim-fout

3832
['1SWASP J1407', '']
2897


In [ ]:
#######################################################################################################################################################
#  In running larger data sets i realized some of the exoplanetary systems don't have any downloadable data
# this is a similar program to the one that looked at Gaia data, out of 2898 initial systems only 2787 systems had data
# It took 5 hours, 39 minutes to run. Ouch
# but it means we can run data sets larger than 50 systems for the actual AI model, which is kinda important
######################################################################################################################################################

import csv
import lightkurve as lk

ExoplanetarySystems = []
FilteredExoplanetSystems = []
badSystemCounter = 0

with open('ExoplanetSystems.csv',newline='') as csvFile:
  reader = csv.DictReader(csvFile)

  for row in reader:
    ExoplanetarySystems.append(row)

print(ExoplanetarySystems[0])
print(len(ExoplanetarySystems))

with open('FilteredExoplanetSystems.csv', 'w', newline='', encoding='UTF8') as csvfile:
  writer = csv.writer(csvfile)
  writer.writerow(["star_name","star_alternate_names"])

  for starName in ExoplanetarySystems:
    sucessfullyFoundData = False
    if starName["star_name"] != "":
      try:
        search_result = lk.search_lightcurve(starName["star_name"])
        data = search_result.download()
        usefulData = data.flux.value
        sucessfullyFoundData = True
      except:
        pass

      if(sucessfullyFoundData):
        print("Writing: ", [starName["star_name"]])
        writer.writerow([starName["star_name"],starName["star_alternate_names"]])
        csvfile.flush()
      else:
        print(starName["star_name"], " Has no data")

print(badSystemCounter)



{'star_name': '1SWASP J1407', 'star_alternate_names': ''}
2897


Could not resolve 1SWASP J1407 to a sky position.
ERROR:lightkurve.search:Could not resolve 1SWASP J1407 to a sky position.
/usr/local/lib/python3.10/dist-packages/lightkurve/search.py:419: LightkurveWarning: Cannot download from an empty search result.
  warnings.warn(


1SWASP J1407  Has no data


/usr/local/lib/python3.10/dist-packages/lightkurve/search.py:424: LightkurveWarning: Warning: 3 files available to download. Only the first file has been downloaded. Please use `download_all()` or specify additional criteria (e.g. quarter, campaign, or sector) to limit your search.
  warnings.warn(


Writing:  ['38 Vir']


...

Could not resolve ZTFJ1637+49 to a sky position.
ERROR:lightkurve.search:Could not resolve ZTFJ1637+49 to a sky position.
/usr/local/lib/python3.10/dist-packages/lightkurve/search.py:419: LightkurveWarning: Cannot download from an empty search result.
  warnings.warn(


ZTFJ1637+49  Has no data


Could not resolve ZTFJ2252-05 to a sky position.
ERROR:lightkurve.search:Could not resolve ZTFJ2252-05 to a sky position.


ZTFJ2252-05  Has no data
0


/usr/local/lib/python3.10/dist-packages/lightkurve/search.py:419: LightkurveWarning: Cannot download from an empty search result.
  warnings.warn(


In [ ]:
######################################################
#1st stage processing of Gaia data, we need to pull out the star names so we can feed them into lightKurve
#Gaia data pulled from https://simbad.cds.unistra.fr/simbad/sim-fout
#output options: file output, max 10,000. Criteria search using something like "Distance.distance < 50"
######################################################
import csv

uniqueStarNameList = []
with open('GaiaDataRaw.csv',newline='') as csvFile:
  reader = csv.reader(csvFile)
  print(type(reader))

  for row in reader:
    #for each row, get first eleem (returning list probably for each column, all data in first cell anyways)
    #split data, grab the 2nd element since that is the system name
    thisLine = row[0].split(";")
    #some lines get index out of range,
    try:
      #print(thisLine[1])
      uniqueStarNameList.append(thisLine[1])
    except:
      pass

#https://www.pythontutorial.net/python-basics/python-write-csv-file/
with open('GaiaStarNames.csv', 'w', newline='', encoding='UTF8') as csvfile:
    writer = csv.writer(csvfile)
    for x in uniqueStarNameList:
      #print(x)
      writer.writerow([x])

print(len(uniqueStarNameList))
for x in range(10):
  print(uniqueStarNameList[x])


In [ ]:
######################################################
#Second analysis stage, go through gaia data and identify the star systems we have lighcurve data available for.
#this can only process like 800-900 stars per hour so I have it set up to limit the number of stars
######################################################
import csv
import lightkurve as lk
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


def readData(startPoint, endPoint):
  GaiaDataWithTransits = []

  with open('GaiaStarNames.csv',newline='') as csvFile:
    reader = csv.reader(csvFile)
    counter = 0
    usefulDataCounter = 0

    with open('Gaia_LC_Systems_0_2k.csv', 'w', newline='', encoding='UTF8') as csvfile:
      writer = csv.writer(csvfile)

      for row in reader:
        if(counter > startPoint and counter < endPoint):
          search_result = None
          #can be trailing whitespace from csv cells, need to remove it so we use .strip()
          search_result = lk.search_lightcurve(row[0].strip())

          try:
            result = search_result.download()
            usefulData = result.flux.value
            usefulDataCounter = usefulDataCounter + 1
            GaiaDataWithTransits.append(row[0].strip())
            #in [] so the data gets saved as one elment not one letter per cell
            writer.writerow([row[0].strip()])
          except:
            pass

          #if(len(search_result.table) > 0):
          #  usefulDataCounter = usefulDataCounter + 1
          #  GaiaDataWithTransits.append(row[0].strip())
          #  #in [] so the data gets saved as one elment not one letter per cell
          #  writer.writerow([row[0].strip()])


        counter= counter + 1
        #print(row[0])

        if(counter % 100 == 0):
          print(counter, "results examined")

      print(counter, usefulDataCounter)

readData(0,2000)

#search_result = lk.search_lightcurve('LP 246-45')
#lc = search_result.download()

#https://docs.lightkurve.org/tutorials/3-science-examples/exoplanets-identifying-transiting-planet-signals.html
#period = np.linspace(1,20,100000)
#bls = lc.to_periodogram(method='bls', period=period, frequency_factor=500)


#lc.plot();
#bls.plot

In [ ]:
######################################################
# Third stage Gaia data processing, we have a set of ~2600 stars we know have exoplanets and a set of
#5000 stars from Gaia and we aren't yet sure there aren't duplicates between the lists
#This will read both lists and check entries against eachother so we can get a full list of 'Positive' and 'negative' systems
#
#  Partway through i realized many of the Exoplanetary systems had been renamed to Kepler-xxx or K2-XX (Kepler mission 2, star system x)
# So i modified the program to check the alternate star names listed in exoplanet.eu_catalog. We went from filtering 2 duplicate systems to 4,
# Which is still not as high as I was initally expecting for an overlap. But we did our due diligence.
######################################################
import random
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Embedding
import lightkurve as lk
import csv


GaiaSystemNames = []
ExoplanetarySystems = []
FilteredGaiaSystems = []

TrainingSystems = []
TrainingYValues = np.array([])
TestSystems = []
TestYValues = np.array([])

Training_LC_Data = np.ndarray([])
Test_LC_Data = np.ndarray([])

maxLen = 0

##############################################################################################################
#   Read Data from our CSV files, verify Gaia data doesn't have duplicate star names from Exoplanet systems
##############################################################################################################
def PrepareData():
  #extract Data from our CSV files
  with open('Gaia_LC_Systems_0_2k.csv',newline='') as csvFile:
      reader = csv.reader(csvFile)
      for row in reader:
        GaiaSystemNames.append(row[0].strip())

  with open('Gaia_LC_Systems_2_4k.csv',newline='') as csvFile:
      reader = csv.reader(csvFile)
      for row in reader:
        GaiaSystemNames.append(row[0].strip())

  with open('FilteredExoplanetSystems.csv',newline='') as csvFile:
    reader = csv.DictReader(csvFile)
    for row in reader:
      #print(row, type(row))
      ExoplanetarySystems.append(row)

  print(len(GaiaSystemNames))

  print(len(ExoplanetarySystems))


  #try to find duplicates and filter them out
  for x in GaiaSystemNames:
    DuplicateName = None

    for y in ExoplanetarySystems:
      if x in y['star_name'] or x in y['star_alternate_names'].split(","):
        DuplicateName = y['star_name'] + " " + y['star_alternate_names']

    if DuplicateName is None:
      FilteredGaiaSystems.append(x)
    else:
      print(x, "is in", DuplicateName)


  print(len(FilteredGaiaSystems))

##############################################################################################################
#  randomly generate test/training groups from the star systems we have identified as candidates.
# We mix systems we know have exoplanets with systems that don't have anything confirmed.
##############################################################################################################
def CreateGroups(NumTrainingSystems, NumTestSystems):
  NegativeGroup = FilteredGaiaSystems[:]
  PositiveGroup = ExoplanetarySystems[:]

  CountPositive = 0
  CountNegative = 0

  global TrainingYValues
  global TestYValues

  #create traning candidate list
  for x in range(NumTrainingSystems):
    WhichGroup = random.randint(0, 1)

    if(WhichGroup) == 0:
      NegativeTarget = random.randint(0, len(NegativeGroup))
      TrainingSystems.append(NegativeGroup.pop(NegativeTarget))
      #TrainingYValues.append(0)

      TrainingYValues = np.append(TrainingYValues,0)

      CountNegative = CountNegative + 1
    else:
      PositiveTarget = random.randint(0, len(PositiveGroup))
      TrainingSystems.append(PositiveGroup.pop(PositiveTarget)['star_name'])
      #TrainingYValues.append(1)
      TrainingYValues = np.append(TrainingYValues,1)

      CountPositive = CountPositive + 1

  print("Training Distribution:",CountPositive,CountNegative)

  CountPositive = 0
  CountNegative = 0

  #create test candidate list
  for x in range(NumTestSystems):
    WhichGroup = random.randint(0, 1)

    if(WhichGroup) == 0:
      NegativeTarget = random.randint(0, len(NegativeGroup))
      TestSystems.append(NegativeGroup.pop(NegativeTarget))
      TestYValues = np.append(TestYValues,0)
      #TestYValues.append(0)
      CountNegative = CountNegative + 1
    else:
      PositiveTarget = random.randint(0, len(PositiveGroup))
      TestSystems.append(PositiveGroup.pop(PositiveTarget)['star_name'])
      TestYValues = np.append(TestYValues,1)
      #TestYValues.append(1)
      CountPositive = CountPositive + 1
  print("Test Distribution:",CountPositive,CountNegative)

##############################################################################################################
#  For the candidate systems we have identified and put into lists we download the lightcurve data
##############################################################################################################
def GetLightKurveData():
  global Training_LC_Data
  global Test_LC_Data
  global maxLen
  tempTrainingList = []
  TempTestList = []


  #format the data into basic list elements
  for x in TrainingSystems:
    searchResult = lk.search_lightcurve(x)
    downloaded_LC_obj = searchResult.download()

    #some data is returning weird/errored results with no flux, may need to refilter the data
    #Until that can happen going to try to proceed to testing basic AI structure
    try:
      data = downloaded_LC_obj.flux.value
      tempTrainingList.append(data)
      if len(data)> maxLen:
        maxLen = len(data)
    except:
      pass


  for y in TestSystems:
    searchResult = lk.search_lightcurve(y)
    downloaded_LC_obj = searchResult.download()

    try:
      data = downloaded_LC_obj.flux.value

      TempTestList.append(data)
      if len(data)> maxLen:
        maxLen = len(data)
    except:
      pass

  print(len(tempTrainingList),tempTrainingList[0])

  #since there is zero garantee that the observations from any two stars will have the same number of data entries
  #we need to padd them with zeros. An embedded mask layer will then work in the AI model to ignore this zeroed data
  #https://www.tensorflow.org/guide/keras/understanding_masking_and_padding

  print(type(tempTrainingList))
  print(np.shape(tempTrainingList))
  Training_LC_Data = tf.keras.utils.pad_sequences(tempTrainingList, maxlen = maxLen, dtype='float32', padding='post')
  Test_LC_Data = tf.keras.utils.pad_sequences(TempTestList, maxlen = maxLen, dtype='float32', padding='post')

  print(np.shape(Training_LC_Data))


##############################################################################################################
#   The AI model doing the actual work. This is set up to work on Binary Classification.
##############################################################################################################

def RunAI():

  #https://www.atmosera.com/blog/binary-classification-with-neural-networks/
  model = Sequential()
  # layers.Embedding(input_dim=maxLen, output_dim=64, mask_zero=True),
  model.add(Embedding(input_dim=maxLen, output_dim=maxLen, mask_zero=True))
  model.add(Dense(512, activation='relu', input_dim=maxLen))
  model.add(Dense(254, activation='relu', input_dim=512))
  model.add(Dense(124, activation='relu', input_dim=254))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  model.build()
  model.summary()

  model.fit(Training_LC_Data, TrainingYValues, batch_size=25, epochs=15, validation_split=0.0)

  score = model.evaluate(Test_LC_Data,TestYValues, verbose=0)


PrepareData()
CreateGroups(70,20)
GetLightKurveData()

print("Length Training Data:",np.shape(Training_LC_Data))
print("Length Training Y Values",np.shape(TrainingYValues))

RunAI()
#for x in Training_LC_Data:
#  print(x)
#print(Test_LC_Data[1])
#print(np.shape(Training_LC_Data))



#https://keras.io/examples/
#https://keras.io/keras_core/api/models/model_training_apis/
#https://keras.io/examples/keras_recipes/debugging_tips/

3403
2786
HD 219134 is in HD 219134 
HD 235088 is in TOI-1430 HD 235088, HIP 98668
LP 141-14 is in WD 1856+534 LP 141-14, TIC 267574918
HD 207897 is in HD 207897 TOI-1611
3399
Training Distribution: 36 34
Test Distribution: 7 13


/usr/local/lib/python3.10/dist-packages/lightkurve/search.py:424: LightkurveWarning: Warning: 18 files available to download. Only the first file has been downloaded. Please use `download_all()` or specify additional criteria (e.g. quarter, campaign, or sector) to limit your search.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/lightkurve/search.py:424: LightkurveWarning: Warning: 9 files available to download. Only the first file has been downloaded. Please use `download_all()` or specify additional criteria (e.g. quarter, campaign, or sector) to limit your search.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/lightkurve/search.py:424: LightkurveWarning: Warning: 14 files available to download. Only the first file has been downloaded. Please use `download_all()` or specify additional criteria (e.g. quarter, campaign, or sector) to limit your search.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/lightkurve/search.py:424: LightkurveWarning: Warning: 30 f

KeyboardInterrupt: ignored